In [2]:
import pandas as pd
import re

# Read the CSV file
df = pd.read_csv('/Users/Magnus/Documents/GitHub/Merge/Journal_Introduction_GenDummy.csv')

# Function to clean the text
def clean_text(text):
    if not isinstance(text, str):
        return text
    
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    
    # Remove the prefix "1. Introduction"
    text = re.sub(r'^1\. Introduction', '', text)
    
    # Remove excessive whitespaces
    text = ' '.join(text.split())
    
    return text

# Apply the cleaning function to the 'Introduction' column
df['Introduction'] = df['Introduction'].apply(clean_text)




In [3]:
print(df)

                                            Introduction  gen_dummy
0      The all-or-nothing mechanism is the most commo...          0
1      A bilateral exchange rate is the price of a cu...          0
2      Performance-contingent bonuses are a prominent...          0
3      A series of theoretical models shows that trad...          0
4      Measuring competitiveness is an important task...          0
...                                                  ...        ...
14297  A trip chain is a series of trips taken in whi...          0
14298  Reliability of public transport (PT) operation...          0
14299  Since the advent of the four-step transport pl...          0
14300  A recent societal trend that made its way into...          0
14301  Urban rail transit systems such as metro is ha...          0

[14302 rows x 2 columns]


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)


2023-08-14 10:01:15.815366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Split your dataframe into training and testing data. You can use sklearn's train_test_split for this
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize your training and testing data
train_encodings = tokenizer.batch_encode_plus(train_df['Introduction'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")
test_encodings = tokenizer.batch_encode_plus(test_df['Introduction'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")



In [8]:


import torch

class GenderDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = GenderDataset(train_encodings, train_df['gen_dummy'].tolist())
test_dataset = GenderDataset(test_encodings, test_df['gen_dummy'].tolist())





In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch.optim as optim

# Initialize BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # num_labels=2 for binary classification

# Define training arguments and initialize Trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    optimizers=(optim.AdamW(model.parameters(), lr=5e-5), None)  # Using PyTorch's AdamW
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_69189/3296796401.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,0.676300


In [5]:
from sklearn.model_selection import train_test_split

# Tokenization
inputs = tokenizer(df['Introduction'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt")

# Split data into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(inputs.input_ids, df['gen_dummy'], test_size=0.2)

# Convert datasets into HuggingFace's format
from datasets import Dataset

train_dataset = Dataset.from_dict({
    'input_ids': train_inputs,
    'labels': train_labels.tolist()
})

val_dataset = Dataset.from_dict({
    'input_ids': val_inputs,
    'labels': val_labels.tolist()
})

# Training settings
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="steps",
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=2e-5,
    output_dir="./results",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train
trainer.train()


KeyboardInterrupt: 

In [ ]:
results = trainer.evaluate()

print(results)
